# Step 8 From Recursive to Iterative

## 8.1 Class Variable (Present state)

Backward function is called inside the backward function **recursively** until it reaches the variable whose creator function does not exist. (self.creator==None)

In [ ]:
class Variable:
    
    def backward(self):
        f = self.creator
        if  f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

## 8.2 Implementing with iterative instead of Recursive

1. call the functions and put them in the list **funcs**
2. if the elements exist in **funcs**, pop out the last element and declare it as f
3. call the input and output of the function and call the backward function
4. if creator exists, put the previous function(creator) inside the list **funcs**. 

In [ ]:
class Variable:
    
    def backward(self):
       funcs = [self.creator]
       while funcs:
           f = funcs.pop()
           x,y=f.input, f.output
           x.grad= f.backward(y.grad)

           if x.creator is not None:
               funcs.append(x.creator)

## 8.3 Testing the code

In [2]:
class Variable:
    
    def __init__(self, data):
        self.data=data
        self.grad=None
        self.creator=None
    
    def set_creator(self, func):
        self.creator=func
    
    def backward(self):
       funcs = [self.creator]
       while funcs:
           f = funcs.pop()
           x,y=f.input, f.output
           x.grad= f.backward(y.grad)

           if x.creator is not None:
               funcs.append(x.creator)

class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y)
        output.set_creator(self) #set creator for the output variable   
        self.input=input #remember the input variable
        self.output=output #also save the output
        return output

class Square(Function):
    def forward(self, x):
        y=x**2
        return y
    
    def backward(self, gy):
        x=self.input.data
        gx=gy*2*x
        return gx
        
class Exp(Function):
    def forward(self, x):
        y=np.exp(x)
        return y
   
    def backward(self, gy):
        x=self.input.data
        gx=np.exp(x)*gy
        return gx

In [4]:
import numpy as np

A= Square()
B= Exp()
C= Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)

y.grad=np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
